# Register SmartSPIM Data To CCF v3.1 Mouse Brain Atlas

## Overview

This notebook demonstrates a reproducible registration pipeline to align downsampled SmartSPIM data to the Allen Mouse Brain Common Coordinate Framework (CCF)$^1$. The notebook can be run interactively to evaluate intermediate results or can be run from start to end with command line parameters.

## Inputs

1. Target (fixed) image. The CCF v3.1 atlas with updated spacing and spatial orientation is available in NIFTI format at http://download.alleninstitute.org/informatics-archive/converted_mouse_ccf/average_template/. More information on the CCF atlas is available at http://help.brain-map.org/display/mouseconnectivity/API

2. Source (moving) image. Stitched SmartSPIM mouse brain images are available on the AWS S3 "aind-open-data" bucket in Zarr format. The largest resolution / smallest image size is used here for performance considerations.

## Outputs

1. ITK multistage composite transform mapping from soure to target space. The composite transform consists of four transform stages.

2. Registered source (moving) image aligned to CCF atlas space.

3. A `summary.txt` file briefly describing the inputs and methods to reproduce registration results.

Other outputs include intermediate registration volumes, parameters, and logs generated by ITKElastix$^2$.

If the notebook is run from the Code Ocean command line with `papermill` then the results directory will also include a copy of the parameterized notebook.

## Assumptions

1. The source and target images are spatially oriented to common anatomical directions. This can be confirmed with a 3D spatial viewer such as 3D Slicer, ITKWidgets, or Neuroglancer. Note that viewers displaying voxel data without spatial information such as matplotlib may produce misleading visuals showing data aligned to different anatomical axes.

2. The source image contains correct metadata. We use tools from `aind_ccf_alignment_experiments` to compose and apply appropriate SmartSPIM metadata.

3. The selected source and target image resolution fit in memory.

## Procedure

1. Data is read in from their respective stores. The CCF atlas is attached via CodeOcean's data attachment mechanism, while the SmartSPIM volume is retrieved directly from S3 with `itk-ioomezarrngff` tooling.

2. An initial translation is constructed to coarsely superimpose the source image on the target image. The source image is updated in place without resampling.

3. The source image is registered to the target image. ITKElastix is used to optimize three transform stages for rigid, affine, and then deformable registration.

4. Results are written out.

## References

1. Quanxin Wang, Song-Lin Ding, Yang Li, Josh Royall, David Feng, Phil Lesnar, Nile Graddis, Maitham Naeemi, Benjamin Facer, Anh Ho, Tim Dolbeare, Brandon Blanchard, Nick Dee, Wayne Wakeman, Karla E. Hirokawa, Aaron Szafer, Susan M. Sunkin, Seung Wook Oh, Amy Bernard, John W. Phillips, Michael Hawrylycz, Christof Koch, Hongkui Zeng, Julie A. Harris, Lydia Ng,
The Allen Mouse Brain Common Coordinate Framework: A 3D Reference Atlas, Cell, Volume 181, Issue 4, 2020, Pages 936-953.e20, ISSN 0092-8674, https://doi.org/10.1016/j.cell.2020.04.007

2. K. Ntatsis, et al. "itk-elastix: Medical image registration in Python." Proceedings of the 22nd Python in Science Conference (SciPy 2023). https://doi.org/10.25080/gerudo-f2bc6f59-00d


## Initialize Notebook

We use [`papermill`](https://papermill.readthedocs.io/en/latest/) to accept command line parameters in the notebook.

In [ ]:
# Default values for command line parameters.
# Refer to https://papermill.readthedocs.io/en/latest/usage-parameterize.html#how-parameters-work

# Typically a path to a stitched SmartSPIM OME-Zarr bucket on Amazon AWS S3
SOURCE_IMAGE_BUCKET = r"s3://aind-open-data/SmartSPIM_652506_2023-01-09_10-18-12_stitched_2023-02-09_17-23-28/processed/OMEZarr/Ex_561_Em_593.zarr"

# Typically a path to the Common Coordinate Framework v3.1 atlas attached to the Code Ocean capsule
# Also available at http://download.alleninstitute.org/informatics-archive/converted_mouse_ccf/average_template/
TARGET_IMAGE_FILEPATH = r"data\CCFv31\intensity\average_template_25.nii.gz"

# The OME-Zarr resolution to use in registration.
# SmartSPIM data is typically scaled down by 2 ^ N in each direction or by 2 ^ 3N in total volume
SOURCE_IMAGE_SCALE = 4

# The isotropic B-spline control grid spacing in mm.
# Configures deformable B-spline registration stage for local alignment.
BSPLINE_GRID_SPACING = 0.5

# The output directory for registration results.
RESULTS_PATH = None

In [ ]:
import sys
import os
import itertools

import itk
import numpy as np

assert "ElastixRegistrationMethod" in dir(
    itk
)  # Ensure itk-elastix is installed

itk.auto_progress(1)

sys.path.append("../src")
from aind_ccf_alignment_experiments.url import (
    parse_smartspim_bucket_path,
    SmartSPIMS3Info,
)
from aind_ccf_alignment_experiments.image import (
    get_physical_size,
    get_sample_bounds,
)
from aind_ccf_alignment_experiments.smartspim import (
    make_smartspim_stream_reader,
)
import aind_ccf_alignment_experiments.registration_methods as registration_methods
from aind_ccf_alignment_experiments.registration_methods import (
    compute_initial_translation,
    register_elastix,
    make_default_elx_parameter_object,
)

In [ ]:
source_info = parse_smartspim_bucket_path(SOURCE_IMAGE_BUCKET)

SAMPLE_NAME = f"{source_info.subject_id}_{source_info.channel_id}"

if not RESULTS_PATH:
    RESULTS_PATH = (
        f"../results/{source_info.subject_id}/{source_info.channel_id}"
    )
os.makedirs(RESULTS_PATH, exist_ok=True)

REGISTERED_IMAGE_OUTPUT_FILEPATH = (
    f"{RESULTS_PATH}/{SAMPLE_NAME}_registered.nii.gz"
)

TRANSFORM_OUTPUT_FILEPATH = f"{RESULTS_PATH}/{SAMPLE_NAME}_transformresult.h5"

print(f"Registration results will be written to {RESULTS_PATH}")

## Load SmartSPIM Image

In [ ]:
smartspim_reader = make_smartspim_stream_reader(
    SOURCE_IMAGE_BUCKET, SOURCE_IMAGE_SCALE
)
smartspim_reader.UpdateLargestPossibleRegion()
source_image = smartspim_reader.GetOutput()

print(source_image)

## Load CCF Atlas Target Image

In [ ]:
target_image = itk.imread(TARGET_IMAGE_FILEPATH, pixel_type=itk.F)

# Note: 3.1 template is in mm (3.0 was um)
print(target_image)

## Validate Data

We briefly evaluate the source and target images to ensure that image physical sizes are on the same order of magnitude as expected. A significant difference in image sizes could indicate a problem with image spacing.

A 3D spatial viewer such as ITKWidgets or Neuroglancer can be used to evaluate that source and target input images share a spatial orientation.

In [ ]:
print(f"CCF physical bounds: {get_sample_bounds(target_image)}")
print(f"SmartSPIM physical bounds: {get_sample_bounds(source_image)}")
print(f"CCF physical size: {get_physical_size(target_image)}")
print(f"SmartSPIM physical size: {get_physical_size(source_image)}")

## Initialize Registration with `itk`

We use tools available in the Insight Toolkit to align the source and target images so that they are initially overlapping in space.

In [ ]:
translation_transform = compute_initial_translation(
    source_image=source_image, target_image=target_image
)

change_information_filter = itk.ChangeInformationImageFilter[
    type(source_image)
].New()
change_information_filter.SetInput(source_image)
change_information_filter.ChangeOriginOn()
change_information_filter.SetOutputOrigin(
    translation_transform.GetInverseTransform().TransformPoint(
        itk.origin(source_image)
    )
)
change_information_filter.Update()

initialized_source_image = change_information_filter.GetOutput()

In [ ]:
print(translation_transform)

In [ ]:
print(initialized_source_image)

In [ ]:
# Verify that the initialized source image bounds overlap with the target image

print(
    f"Original input source image bounds: {get_sample_bounds(source_image)[0]}, {get_sample_bounds(source_image)[1]}"
)
print(
    f"Translated source image bounds: {get_sample_bounds(initialized_source_image)[0]}, {get_sample_bounds(initialized_source_image)[1]}"
)
print(
    f"Target image bounds: {get_sample_bounds(target_image)[0]}, {get_sample_bounds(target_image)[1]}"
)

In [ ]:
# itkwidgets.compare_images(initialized_source_image, target_image)

## Register with `itk-elastix`

We use the tools developed in Elastix and made available via the ITKElastix Python module to perform multistage registration.

In [ ]:
parameter_object = make_default_elx_parameter_object()
print(parameter_object)

In [ ]:
bspline_map = parameter_object.GetParameterMap(2)
bspline_map["FinalGridSpacingInPhysicalUnits"] = (
    f"{BSPLINE_GRID_SPACING:0.6f}",
)
parameter_object.SetParameterMap(2, bspline_map)
print(parameter_object)

In [ ]:
(
    composite_transform,
    registered_source_image,
    registration_method,
) = register_elastix(
    source_image=initialized_source_image,
    target_image=target_image,
    parameter_object=parameter_object,
    log_filepath=f"{RESULTS_PATH}/elastix-output.txt",
    verbose=True,
)

In [ ]:
# Verify that the registered source image bounds concide with the target image

print(
    f"Registered source image bounds: {get_sample_bounds(registered_source_image)[0]},"
    f"{get_sample_bounds(registered_source_image)[1]}"
)
print(
    f"Target image bounds: {get_sample_bounds(target_image)[0]}, {get_sample_bounds(target_image)[1]}"
)

In [ ]:
print(composite_transform)

## Save Outputs To Disk

Reproducible results should be saved to the capsule 'data' folder. Registration results from this notebook include:
- The registered, resampled SmartSPIM image. This can be compared with the target CCF average template image or CCF label atlas in a spatial viewer for visual evaluation of registration fitness.
- The sequence of transforms used to map from the source SmartSPIM sample space to target CCF space. We can map corresponding information in SmartSPIM source space such as segmentations or other markups into CCF space by applying this sequence of transformations.



In [ ]:
composite_transform.PrependTransform(translation_transform)

In [ ]:
print(composite_transform)

In [ ]:
itk.transformwrite(
    [composite_transform], TRANSFORM_OUTPUT_FILEPATH, compression=True
)

itk.imwrite(
    registered_source_image,
    REGISTERED_IMAGE_OUTPUT_FILEPATH,
    compression=True,
)

In [ ]:
# Write a summary file for reproducibility

with open(f"{RESULTS_PATH}/summary.txt", "w") as f:
    f.write(f"SUBJECT_ID {source_info.subject_id}\n")
    f.write(f"SAMPLE_CHANNEL {source_info.channel_id}\n")
    f.write(f"SAMPLE_LEVEL {SOURCE_IMAGE_SCALE}\n")
    f.write(f"SAMPLE_FILEPATH {source_info.bucket_path}\n")
    f.write(f"TARGET_FILEPATH {TARGET_IMAGE_FILEPATH}\n")
    f.write("\n")
    f.write(f"INITIALIZATION_METHOD TargetToSourceMidpoint\n")
    f.write(f"TRANSFORM_METHOD ELASTIX\n")
    f.write(str(parameter_object))
    f.write("\n")

    f.write(f"OUTPUT_TRANSFORM_FILEPATH {TRANSFORM_OUTPUT_FILEPATH}\n")
    f.write(f"OUTPUT_SAMPLE_FILEPATH {REGISTERED_IMAGE_OUTPUT_FILEPATH}\n")